# Blog Automation

In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
load_dotenv()

True

### Defining LLMs and Prompts

In [3]:
drafting_llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview"
) 

content_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)

eval_llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash"
)

In [4]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    eval_score: float

In [5]:
def draft_outline(state: BlogState) -> BlogState:
    title = state['title']
    
    prompt = f"Write a detailed outline for the given topic. Topic: {title}" 
    
    response = drafting_llm.invoke(prompt).content
    
    if isinstance(response, list):
        outline = response[0]['text']
    else:
        outline = response
    
    state['outline'] = outline
    
    return state

In [6]:
def create_content(state: BlogState) -> BlogState:
    title = state['title']
    
    prompt = f'Write a detailed blog post with proper headings, sub-headings, paragraphs and bullets on the title: {title} according to the outline: {state["outline"]}' 
    
    content = content_llm.invoke(prompt).content
    
    state['content'] = content
    
    return state

In [7]:
import re

def evaluate_content(state: BlogState) -> BlogState:
    prompt = f"""Rate my Blog 'Content' According to the given 'title' and 'outline' in a floating number ranging from 0.0 to 10.0.
             Title: {state['title']}, 
             Outline: {state['outline']}
             Content: {state['content']}
             
             IMPORTANT: Respond with ONLY the numeric score (e.g., 7.5). No other text.
    """ 
    
    eval_response = eval_llm.invoke(prompt).content
    
    match = re.search(r'(\d+\.?\d*)', str(eval_response))
    if match:
        state['eval_score'] = float(match.group(1))
    else:
        state['eval_score'] = 0.0 
    
    return state

### Create Graph

In [8]:
graph = StateGraph(BlogState)

graph.add_node('draft_outline', draft_outline)
graph.add_node('create_content', create_content)
graph.add_node('evaluate_content', evaluate_content)

graph.add_edge(START, 'draft_outline')
graph.add_edge('draft_outline', 'create_content')
graph.add_edge('create_content', 'evaluate_content')
graph.add_edge('evaluate_content', END)

workflow = graph.compile()

In [9]:
initial_state = {
    'title': 'Rise of AI in Pakistan'
}

final_state = workflow.invoke(initial_state)


In [10]:
print(final_state['outline']) 

This detailed outline explores the emergence, growth, current status, and future prospects of Artificial Intelligence (AI) in Pakistan, covering everything from government policy to grassroots innovation.

---

# Outline: The Rise of AI in Pakistan

## I. Introduction
*   **A. Global Context:** Briefly mention the 4th Industrial Revolution (4IR) and AI as a global economic driver.
*   **B. Defining the Pakistani Landscape:** Pakistan’s transition from a traditional IT services provider to an emerging hub for AI development.
*   **C. Thesis Statement:** While Pakistan faces infrastructural and economic hurdles, its youthful demographic, government initiatives, and expanding startup ecosystem are positioning AI as a cornerstone of its digital future.

## II. Historical Context and Evolution
*   **A. The Early 2000s:** Initial focus on basic software development and IT outsourcing.
*   **B. The Data Shift (2015–2020):** Transition from general software engineering to Data Science and Mach

In [11]:
print(final_state['content']) 

## The Digital Frontier: Unpacking the Rise of AI in Pakistan

The world is in the throes of the Fourth Industrial Revolution (4IR), a transformative era characterized by the fusion of technologies that blur the lines between the physical, digital, and biological spheres. At its heart lies Artificial Intelligence (AI), a potent economic driver reshaping industries, economies, and societies globally. Pakistan, historically recognized for its burgeoning IT services sector, is now strategically repositioning itself. No longer content with merely providing traditional IT outsourcing, the nation is rapidly emerging as a dynamic hub for AI development and innovation.

This blog post delves into the intricate journey of AI in Pakistan. While the country grapples with significant infrastructural and economic hurdles, its vast youthful demographic, proactive government initiatives, and an increasingly vibrant startup ecosystem are collectively propelling AI to become a foundational cornerstone 

In [12]:
print(final_state['eval_score'])

9.8
